In [ ]:
import pandas as pd
from windowing import GeneData
pd.set_option('display.max_columns', 500)

data = pd.read_csv('data/22-23/player_data.csv')
data = data[data['minutes'] > 0].reset_index(drop=True)
gene = GeneData(data)

gene.reset_data()
gene.add_team_performance()
gene.position_filter('MID')
gene.window_split()
gene.scale_data()
gene.wrangle_data()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

Tabular Model

In [ ]:
model = Sequential()
model.add(Dense(8, input_dim=dim, activation="relu"))
model.add(Dense(4, activation="relu"))

In [ ]:
gene.train_data['player_data'].shape

